# Transcribe 2 hours of audio in less than 2 minutes with Whisper


This tutorial demonstrates how to use the `pruna` package to optimize any custom whisper model. In this case, the smash function wraps the model into an efficient pipeline, which will transcribe 2 hours of audio in under 2 minutes on an A100 GPU We will use the `openai/whisper-large-v3` model as an example.

In [1]:
# if you are not running the latest version of this tutorial, make sure to install the matching version of pruna
# the following command will install the latest version of pruna

%pip install pruna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.0/131.0 kB 6.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 31.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of llmcompressor to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.2/207.2 kB 19.3 MB/

### 1. Loading the ASR model

First, load your ASR model.

In [1]:
import torch
from transformers import AutoModelForSpeechSeq2Seq

device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id_whisper = "openai/whisper-large-v3"

model_whisper = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id_whisper, torch_dtype=torch_dtype, use_safetensors=True, low_cpu_mem_usage=True,
)
model_whisper.to(device)

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(128, 1280, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 1280)
      (layers): ModuleList(
        (0-31): 32 x WhisperEncoderLayer(
          (self_attn): WhisperSdpaAttention(
            (k_proj): Linear(in_features=1280, out_features=1280, bias=False)
            (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1280, out_features=5120, bias=True)
          (fc2): Linear(in_features=5120, out_features=1280, bia

In [8]:
import torch
from transformers import AutoModelForSpeechSeq2Seq

device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "unsloth/whisper-large-v3-turbo"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, use_safetensors=True, low_cpu_mem_usage=True,
)
model.to(device)

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(128, 1280, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 1280)
      (layers): ModuleList(
        (0-31): 32 x WhisperEncoderLayer(
          (self_attn): WhisperSdpaAttention(
            (k_proj): Linear(in_features=1280, out_features=1280, bias=False)
            (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1280, out_features=5120, bias=True)
          (fc2): Linear(in_features=5120, out_features=1280, bia

### 2. Initializing the Smash Config

Next, initialize the smash_config. Since the compiler require a processor, we add it to the smash_config.

In [2]:
from pruna import SmashConfig

# Initialize the SmashConfig
smash_config = SmashConfig()
smash_config.add_tokenizer(model_id_whisper)
smash_config.add_processor(model_id_whisper)
smash_config['compiler'] = 'c_whisper'
smash_config['batcher'] = 'whisper_s2t'
# uncomment the following line to quantize the model to 8 bits
# smash_config['c_whisper_weight_bits'] = 8

Multiple distributions found for package optimum. Picked distribution: optimum-quanto
INFO - Using best available device: 'cuda'


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

normalizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

In [9]:
from pruna import SmashConfig
import os
from transformers import AutoTokenizer, AutoProcessor

# Create directory with complete tokenizer files
complete_model_path = "./unsloth-whisper-complete"
os.makedirs(complete_model_path, exist_ok=True)

# Download tokenizer from OpenAI (has complete files including tokenizer.json)
tokenizer = AutoTokenizer.from_pretrained("openai/whisper-large-v3-turbo")
processor = AutoProcessor.from_pretrained("openai/whisper-large-v3-turbo")

# Save complete tokenizer files locally
tokenizer.save_pretrained(complete_model_path)
processor.save_pretrained(complete_model_path)


# Initialize the SmashConfig
smash_config = SmashConfig()
smash_config.add_tokenizer(complete_model_path)
smash_config.add_processor(complete_model_path)
smash_config['compiler'] = 'c_whisper'
smash_config['batcher'] = 'whisper_s2t'
# uncomment the following line to quantize the model to 8 bits
# smash_config['c_whisper_weight_bits'] = 8

INFO - Using best available device: 'cuda'


### 3. Smashing the Model

Now, smash the model. This will take approximately 2 minutes on a T4 GPU.

In [10]:
from pruna import smash

# Smash the model
smashed_model = smash(
    model=model,
    smash_config=smash_config,
)

INFO - Starting compiler c_whisper...
INFO - compiler c_whisper was applied successfully.
INFO - Starting batcher whisper_s2t...
INFO - Preparing model for inference with batch size 1...
INFO - batcher whisper_s2t was applied successfully.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [3]:
from pruna import smash

# Smash the model
smashed_model = smash(
    model=model_whisper,
    smash_config=smash_config,
)

INFO - Starting compiler c_whisper...
INFO - compiler c_whisper was applied successfully.
INFO - Starting batcher whisper_s2t...
INFO - Preparing model for inference with batch size 1...
INFO - batcher whisper_s2t was applied successfully.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


### 4. Preparing the Input

In [5]:
import requests

response = requests.get("https://huggingface.co/datasets/reach-vb/random-audios/resolve/main/sam_altman_lex_podcast_367.flac")
audio_sample = 'sam_altman_lex_podcast_367.flac'

# Save the content to the specified file
with open(audio_sample, 'wb') as f:
    f.write(response.content)

### 5. Running the Model

Finally, run the model to transcribe the audio file. Make sure you have `ffmpeg` installed.

In [11]:
# Display the result
smashed_model(audio_sample)

Transcribing: 100%|██████████| 100/100 [02:53<00:00,  1.73s/it]


'We have been a misunderstood and badly mocked org for a long time. Like when we started, we like announced the org at the end of 2015 and said we were going to work on AGI. Like people thought we were batshit insane. Yeah. You know, like I remember at the time a eminent AI scientist at a large industrial AI lab was DMing individual reporters being like, these people aren\'t very good and it\'s ridiculous to talk about AGI and I can\'t believe you\'re giving them time of day. And it\'s like, that was the level of pettiness and rancor in the field at a new group of people saying, we\'re gonna try to build AGI. So OpenAI and DeepMind was a small collection of folks who were brave enough to talk about AGI in the face of mockery. We don\'t get mocked as much now. Don\'t get mocked as much now. The following is a conversation with Sam Altman, CEO of OpenAI, the company behind GPT-4, JAD-GPT, DALI, Codex, and many other technologies, which both individually and together constitute some of th

### Wrap Up

Congratulations! You have successfully smashed an ASR model. You can now use the `pruna` package to optimize any custom ASR model. The only parts that you should modify are step 1, 4 and 5 to fit your use case.